# Status calculation with ekostat_calculator

In [12]:
import os
import sys
import datetime
import core

In [13]:
print(os.getcwd())

D:\github\ekostat_calculator


-----------------------------------
## Select 
### Directories and file paths

In [14]:
if __name__ == '__main__':
    nr_marks = 60
    print('='*nr_marks)
    print('Running module "lv_test_file.py"')
    print('-'*nr_marks)
    print('')
    
    #root_directory = os.path.dirname(os.path.abspath(__file__)) # works in 
    root_directory = os.getcwd() # works in notebook
    resources_directory = root_directory + '/resources'
    filter_directory = root_directory + '/workspaces/default/filters'
    data_directory = root_directory + '/workspaces/default/data'
#    est_core.StationList(root_directory + '/test_data/Stations_inside_med_typ_attribute_table_med_delar_av_utsjö.txt')
    core.ParameterList()
    
    #--------------------------------------------------------------------------
    
    print('{}\nSet directories and file paths'.format('*'*nr_marks))
    raw_data_file_path = data_directory + '/raw_data/data_BAS_2000-2009.txt'
    first_filter_data_directory = data_directory + '/filtered_data' 
    
    first_data_filter_file_path = filter_directory + '/selection_filters/first_data_filter.txt' 
    winter_data_filter_file_path = filter_directory + '/selection_filters/winter_data_filter.txt'
    summer_data_filter_file_path = filter_directory + '/selection_filters/summer_data_filter.txt'
    
    tolerance_filter_file_path = filter_directory + '/tolerance_filters/tolerance_filter_template.txt'

Running module "lv_test_file.py"
------------------------------------------------------------

************************************************************
Set directories and file paths


### Set up filters
TODO: Store selection filters as attributes or something that allows us to not have to call them before calculating indicators/qualityfactors

In [15]:
    print('{}\nInitiating filters'.format('*'*nr_marks))
    first_filter = core.DataFilter('First filter', file_path = first_data_filter_file_path)
    winter_filter = core.DataFilter('winter_filter', file_path = winter_data_filter_file_path)
    winter_filter.save_filter_file(filter_directory + '/selection_filters/winter_data_filter_save.txt') # mothod available
    summer_filter = core.DataFilter('summer_filter', file_path = summer_data_filter_file_path)
    summer_filter.save_filter_file(filter_directory + '/selection_filters/summer_data_filter_save.txt') # mothod available
    tolerance_filter = core.ToleranceFilter('test_tolerance_filter', file_path = tolerance_filter_file_path)
    print('done\n{}.'.format('*'*nr_marks))

************************************************************
Initiating filters
done
************************************************************.


### Load reference values

In [17]:
    print('{}\nLoading reference values'.format('*'*nr_marks))
    core.RefValues()
    core.RefValues().add_ref_parameter_from_file('DIN_winter', resources_directory + '/classboundaries/nutrients/classboundaries_din_vinter.txt')
    core.RefValues().add_ref_parameter_from_file('TOTN_winter', resources_directory + '/classboundaries/nutrients/classboundaries_totn_vinter.txt')
    core.RefValues().add_ref_parameter_from_file('TOTN_summer', resources_directory + '/classboundaries/nutrients/classboundaries_totn_summer.txt')
    print('done\n{}.'.format('*'*nr_marks))

************************************************************
Loading reference values
done
************************************************************.


------------------------------------------------------
## Data
### Select data and create DataHandler instance

In [18]:
    # Handler (raw data)
    raw_data = core.DataHandler('raw')
    raw_data.add_txt_file(raw_data_file_path, data_type='column') 

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2827: DtypeWarning: Columns (16,28,32,34,36,38,40,44,46,48,54,56,58,60,62,72,74,76,78,103) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


### Apply filters to selected data

In [19]:
    # Use first filter 
    filtered_data = raw_data.filter_data(first_filter) 
    
    # Save filtered data (first filter) as a test
    filtered_data.save_data(first_filter_data_directory)
    
    # Load filtered data (first filter) as a test
    loaded_filtered_data = core.DataHandler('first_filtered')
    loaded_filtered_data.load_data(first_filter_data_directory)

-----------------------------------------------------
## Calculate Quality factors
### Create an instance of NP Qualityfactor class

In [20]:
    qf_NP = core.QualityFactorNP()
    # use set_data_handler to load the selected data to the QualityFactor
    qf_NP.set_data_handler(data_handler = loaded_filtered_data)

Class IndicatorBase:
No attribute for parameter DIN  'NoneType' object has no attribute 'set_data_handler'


True

### Filter parameters in QualityFactorNP
THIS SHOULD BE DEFAULT

In [21]:
    print('{}\nApply season filters to parameters in QualityFactor\n'.format('*'*nr_marks))
    # First general filter 
    qf_NP.filter_data(data_filter_object = first_filter) 
    # winter filter
    qf_NP.filter_data(data_filter_object = winter_filter, indicator = 'TOTN_winter') 
    qf_NP.filter_data(data_filter_object = winter_filter, indicator = 'DIN_winter')
    # summer filter
    qf_NP.filter_data(data_filter_object = summer_filter, indicator = 'TOTN_summer')
    print('done\n{}.'.format('*'*nr_marks))

************************************************************
Apply season filters to parameters in QualityFactor

Class IndicatorDIN:
No attribute for parameter DIN
Error message: 'NoneType' object has no attribute 'filter_data'
	Calculating DIN...
			...Done.
	Calculating DIN...
			...Done.
done
************************************************************.


### Calculate Quality Factor EQR

In [22]:
    print('{}\nApply tolerance filters to all indicators in QualityFactor and get result\n'.format('*'*nr_marks))
    qf_NP.get_EQR(tolerance_filter)  

************************************************************
Apply tolerance filters to all indicators in QualityFactor and get result

		Calculating Indicator EK values.....
			get_ref_value_for_par_with_salt_ref for indicator DIN_winter...
			Calculate DIN_winter Ek value...
			DIN_winter Ek value Calculated
			get_ref_value_for_par_with_salt_ref for indicator TOTN_winter...
			Calculate TOTN_winter Ek value...
			TOTN_winter Ek value Calculated
			get_ref_value_for_par_with_salt_ref for indicator TOTN_summer...
			Calculate TOTN_summer Ek value...
			TOTN_summer Ek value Calculated
		Indicator EK values calculated


In [23]:
    print(qf_NP.class_result)
    print('-'*nr_marks)
    print('done')
    print('-'*nr_marks)

{'qualityfactor': 'Nutrients', 'type_area': None, 'status': 'High', 'EQR': 0.80175571329992934, 'qf_EQR': {'qf_NP_winter': 3.8422477622480913, 'qf_NP_summer': 4.1753093707512017, 'qf_NP_EQR': 4.0087785664996467}, 'all_ok': False}
------------------------------------------------------------
done
------------------------------------------------------------
